# Kuan Chen Chen Milestone 2

## import library

In [299]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import json
import requests
import io
import seaborn as sns
import re
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV




from matplotlib import pyplot as plt
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go


## Dataset manipulate

### Scaler & Labeel

#### Overall scaler

In [110]:
raw_data = pd.read_csv("all_sports_data.csv")
raw_data.describe(),raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15560 entries, 0 to 15559
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       15560 non-null  object 
 1   overall    15560 non-null  int64  
 2   height_cm  15560 non-null  int64  
 3   weight_kg  15560 non-null  int64  
 4   Age        15560 non-null  int64  
 5   years_pro  2028 non-null   float64
 6   position   15560 non-null  object 
 7   APY        15560 non-null  float64
 8   sport      15560 non-null  object 
dtypes: float64(2), int64(4), object(3)
memory usage: 1.1+ MB


(            overall     height_cm     weight_kg           Age    years_pro  \
 count  15560.000000  15560.000000  15560.000000  15560.000000  2028.000000   
 mean      67.139524    182.639203     79.429177     25.456298     3.856509   
 std        7.903444      7.823814     15.025329      4.650786     3.443049   
 min       26.000000    156.000000     49.000000     16.000000     0.000000   
 25%       62.000000    177.000000     71.000000     22.000000     1.000000   
 50%       67.000000    183.000000     76.000000     25.000000     3.000000   
 75%       72.000000    188.000000     83.000000     29.000000     6.000000   
 max       99.000000    226.000000    172.000000     43.000000    22.000000   
 
                 APY  
 count  1.556000e+04  
 mean   1.261962e+06  
 std    3.972747e+06  
 min    0.000000e+00  
 25%    5.200000e+04  
 50%    2.080000e+05  
 75%    9.360000e+05  
 max    6.000000e+07  ,
 None)

In [111]:
data_scaled = raw_data.copy()
mm_scaler = MinMaxScaler()
ss_scaler = StandardScaler()

for sport in ["NFL", "NBA","FIFA"]:
    sport_mask = data_scaled["sport"] == sport
    sport_indices = data_scaled[sport_mask].index
    sport_overall = data_scaled.loc[sport_mask, "overall"].values.reshape(-1, 1)
    
    minmax = mm_scaler.fit_transform(sport_overall)
    data_scaled.loc[sport_indices, "overall_minmax"] = minmax.flatten() 
    
    standard = ss_scaler.fit_transform(sport_overall)
    data_scaled.loc[sport_indices, "overall_standard"] = standard.flatten()  


data_scaled

,name,overall,height_cm,weight_kg,Age,years_pro,position,APY,sport,overall_minmax,overall_standard
0,Dak Prescott,87,188,108,30,7.0,QB,60000000.0,NFL,0.835616,1.582078
1,Trevor Lawrence,82,198,100,23,2.0,QB,55000000.0,NFL,0.767123,1.082987
2,Jordan Love,70,193,99,24,3.0,QB,55000000.0,NFL,0.602740,-0.114831
3,Joe Burrow,95,193,98,26,3.0,QB,55000000.0,NFL,0.945205,2.380623
4,Tua Tagovailoa,83,185,98,25,3.0,QB,53100000.0,NFL,0.780822,1.182805
...,...,...,...,...,...,...,...,...,...,...,...
15555,Umut Yaşar Keçeci,58,182,73,19,NaN,ST,104000.0,FIFA,0.250000,-1.150147
15556,Rakan Al Kaabi,54,174,71,20,NaN,"CDM, CM",104000.0,FIFA,0.159091,-1.702407
15557,Rodrigo Frutos,58,188,87,20,NaN,GK,26000.0,FIFA,0.250000,-1.150147
15558,Christian Bos,55,180,70,18,NaN,RB,26000.0,FIFA,0.181818,-1.564342


#### Data label

In [164]:
for sport in data_scaled["sport"].unique():
    positions = data_scaled[data_scaled["sport"] == sport]["position"].unique()
    print(f"\n{sport} have {len(positions)} positions:")
    print(positions)


NFL have 18 positions:
['QB' 'WR' 'EDGE' 'IDL' 'RT' 'LT' 'CB' 'LG' 'RG' 'LB' 'RB' 'S' 'C' 'TE'
 'K' 'FB' 'P' 'LS']

NBA have 5 positions:
['PF' 'PG' 'SF' 'SG' 'C']

FIFA have 648 positions:
['CB' 'ST' 'GK' 'CM, RB' 'LW, RW, CAM' 'ST, CAM, LW' 'CM' 'ST, CF' 'LB'
 'RB, RM' 'RWB, RB' 'CF, ST, CAM' 'LB, RB, LM' 'CAM, ST' 'LM, RM, CAM'
 'CDM, CM, CB' 'CM, CDM' 'CAM, RM' 'CF, CAM' 'RB, CB' 'CM, CAM, ST'
 'CM, CAM' 'LB, LWB' 'CF, ST' 'ST, LW, RM' 'LM, RM' 'CDM, CM' 'RW, LW, ST'
 'CB, CDM' 'RM, LM' 'RB, RWB, RM' 'RW, CAM' 'ST, CAM' 'CDM' 'RB, CM'
 'CB, LWB' 'CB, RB, LB' 'CAM, CF' 'CAM' 'CM, CDM, CAM' 'CF, RM, CM'
 'LM, LW, CAM' 'LB, CB' 'CAM, LM, RM' 'RB, LB, CB' 'RB, LB' 'CM, LW, ST'
 'ST, LM, CAM' 'CAM, CM' 'CM, CDM, ST' 'CDM, CB, CM' 'LB, RB, CB' 'LM'
 'ST, LM' 'RWB' 'CDM, LB, CM' 'CAM, CM, CDM' 'ST, LW, RW' 'LB, RB'
 'RB, RWB' 'CB, CDM, LB' 'CB, CDM, CM' 'LW, LWB' 'RB, RM, RW'
 'ST, CAM, CM' 'CAM, ST, RW' 'CB, RB' 'CB, RB, RM' 'LM, LW' 'RM, RW, LM'
 'RM, ST, LM' 'CDM, CB' 'CB, RB, CDM' 'L

In [165]:
unique_positions = data_scaled["position"].unique()
position_count = len(unique_positions)
position_count

667

In [229]:
def transform_encoding(data,column, nfl_map, nba_map, new_column_name):
    
    data_simplified = data.copy()
    def apply_position_rule(row):
        if row['sport'] == 'NFL':
            return nfl_map.get(row[column], row[column])
        elif row['sport'] == 'NBA':
            return nba_map.get(row[column], row[column])
        elif row['sport'] == 'FIFA':
            return 0
        else:
            return row['position']  
    
    data_simplified[new_column_name] = data_simplified.apply(apply_position_rule, axis=1)
    return data_simplified


In [230]:
def check_encode(data,label):
    for sport in data['sport'].unique():
        positions = data[data['sport'] == sport][label].unique()
        print(f"\n{sport} have {len(positions)} positions:")
        print(positions)

In [231]:
nfl_18to18_label = {
        'QB': 1,
        'RB': 2,
        'FB': 3 ,
        'WR': 4,
        'TE': 5,
        'LT': 6,
        'RT': 7,
        'LG': 8,
        'RG': 9,
        'C': 10,
        'EDGE': 11,
        'IDL': 12,
        'LB': 13,
        'CB': 14,
        'S': 15,
        'K': 16,
        'P': 17,
        'LS': 18
    }

In [232]:
nba_5to5_lable = {
    'PF': 4, 
    'PG': 1,
    'SF': 3 ,
    'SG': 2,
    'C': 5
}

In [233]:
data_label = transform_encoding(data_scaled,"position", nfl_18to18_label, nba_5to5_lable , "position_label")
data_label 

,name,overall,height_cm,weight_kg,Age,years_pro,position,APY,sport,overall_minmax,overall_standard,position_label
0,Dak Prescott,87,188,108,30,7.0,QB,60000000.0,NFL,0.835616,1.582078,1
1,Trevor Lawrence,82,198,100,23,2.0,QB,55000000.0,NFL,0.767123,1.082987,1
2,Jordan Love,70,193,99,24,3.0,QB,55000000.0,NFL,0.602740,-0.114831,1
3,Joe Burrow,95,193,98,26,3.0,QB,55000000.0,NFL,0.945205,2.380623,1
4,Tua Tagovailoa,83,185,98,25,3.0,QB,53100000.0,NFL,0.780822,1.182805,1
...,...,...,...,...,...,...,...,...,...,...,...,...
15555,Umut Yaşar Keçeci,58,182,73,19,NaN,ST,104000.0,FIFA,0.250000,-1.150147,0
15556,Rakan Al Kaabi,54,174,71,20,NaN,"CDM, CM",104000.0,FIFA,0.159091,-1.702407,0
15557,Rodrigo Frutos,58,188,87,20,NaN,GK,26000.0,FIFA,0.250000,-1.150147,0
15558,Christian Bos,55,180,70,18,NaN,RB,26000.0,FIFA,0.181818,-1.564342,0


In [234]:
check_encode(data_label,"position_label")


NFL have 18 positions:
[ 1  4 11 12  7  6 14  8  9 13  2 15 10  5 16  3 17 18]

NBA have 5 positions:
[4 1 3 2 5]

FIFA have 1 positions:
[0]


#### Position decrease

In [235]:
nfl_18to8_map = {
        'QB': 'QB',
        'RB': 'RB',
        'FB': 'RB',
        'WR': 'Receiver',
        'TE': 'Receiver',
        'LT': 'OL',
        'RT': 'OL',
        'LG': 'OL',
        'RG': 'OL',
        'C': 'OL',
        'EDGE': 'DL',
        'IDL': 'DL',
        'LB': 'LB',
        'CB': 'DB',
        'S': 'DB',
        'K': 'Specialist',
        'P': 'Specialist',
        'LS': 'Specialist'
    }

In [236]:
nba_5to3_map = {
    'PF':'F', 
    'PG':'G',
    'SF':'F',
    'SG':'G',
    'C':'C'
}

In [237]:
data_encode = transform_encoding(data_label,"position",nfl_18to8_map,nba_5to5_map,"position_simplified")

In [238]:
check_encode(data_encode,'position_simplified' )


NFL have 8 positions:
['QB' 'Receiver' 'DL' 'OL' 'DB' 'LB' 'RB' 'Specialist']

NBA have 3 positions:
['F' 'G' 'C']

FIFA have 1 positions:
[0]


In [239]:
data_encode

,name,overall,height_cm,weight_kg,Age,years_pro,position,APY,sport,overall_minmax,overall_standard,position_label,position_simplified
0,Dak Prescott,87,188,108,30,7.0,QB,60000000.0,NFL,0.835616,1.582078,1,QB
1,Trevor Lawrence,82,198,100,23,2.0,QB,55000000.0,NFL,0.767123,1.082987,1,QB
2,Jordan Love,70,193,99,24,3.0,QB,55000000.0,NFL,0.602740,-0.114831,1,QB
3,Joe Burrow,95,193,98,26,3.0,QB,55000000.0,NFL,0.945205,2.380623,1,QB
4,Tua Tagovailoa,83,185,98,25,3.0,QB,53100000.0,NFL,0.780822,1.182805,1,QB
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,Umut Yaşar Keçeci,58,182,73,19,NaN,ST,104000.0,FIFA,0.250000,-1.150147,0,0
15556,Rakan Al Kaabi,54,174,71,20,NaN,"CDM, CM",104000.0,FIFA,0.159091,-1.702407,0,0
15557,Rodrigo Frutos,58,188,87,20,NaN,GK,26000.0,FIFA,0.250000,-1.150147,0,0
15558,Christian Bos,55,180,70,18,NaN,RB,26000.0,FIFA,0.181818,-1.564342,0,0


In [240]:
nfl_8to8_label = {
    'QB': 1,
    'Receiver': 2,
    'DL': 3,
    'OL': 4,
    'DB': 5,
    'LB': 6,
    'RB': 7,
    'Specialist': 8
    }

In [241]:
nba_3to3_label = {
    'F':2, 
    'G':1,
    'C':3
}

In [242]:
df = transform_encoding(data_encode,'position_simplified',nfl_8to8_label,nba_3to3_label,"position_encode_simplified")
check_encode(df,"position_encode_simplified")


NFL have 8 positions:
[1 2 3 4 5 6 7 8]

NBA have 3 positions:
[2 1 3]

FIFA have 1 positions:
[0]


### Data split

In [243]:
df_no_years = df.drop(labels="years_pro",  axis=1)
df_no_years 

,name,overall,height_cm,weight_kg,Age,position,APY,sport,overall_minmax,overall_standard,position_label,position_simplified,position_encode_simplified
0,Dak Prescott,87,188,108,30,QB,60000000.0,NFL,0.835616,1.582078,1,QB,1
1,Trevor Lawrence,82,198,100,23,QB,55000000.0,NFL,0.767123,1.082987,1,QB,1
2,Jordan Love,70,193,99,24,QB,55000000.0,NFL,0.602740,-0.114831,1,QB,1
3,Joe Burrow,95,193,98,26,QB,55000000.0,NFL,0.945205,2.380623,1,QB,1
4,Tua Tagovailoa,83,185,98,25,QB,53100000.0,NFL,0.780822,1.182805,1,QB,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,Umut Yaşar Keçeci,58,182,73,19,ST,104000.0,FIFA,0.250000,-1.150147,0,0,0
15556,Rakan Al Kaabi,54,174,71,20,"CDM, CM",104000.0,FIFA,0.159091,-1.702407,0,0,0
15557,Rodrigo Frutos,58,188,87,20,GK,26000.0,FIFA,0.250000,-1.150147,0,0,0
15558,Christian Bos,55,180,70,18,RB,26000.0,FIFA,0.181818,-1.564342,0,0,0


In [244]:
df_pro_years = df[df["sport"].isin(["NFL", "NBA"])]
df_pro_years

,name,overall,height_cm,weight_kg,Age,years_pro,position,APY,sport,overall_minmax,overall_standard,position_label,position_simplified,position_encode_simplified
0,Dak Prescott,87,188,108,30,7.0,QB,60000000.0,NFL,0.835616,1.582078,1,QB,1
1,Trevor Lawrence,82,198,100,23,2.0,QB,55000000.0,NFL,0.767123,1.082987,1,QB,1
2,Jordan Love,70,193,99,24,3.0,QB,55000000.0,NFL,0.602740,-0.114831,1,QB,1
3,Joe Burrow,95,193,98,26,3.0,QB,55000000.0,NFL,0.945205,2.380623,1,QB,1
4,Tua Tagovailoa,83,185,98,25,3.0,QB,53100000.0,NFL,0.780822,1.182805,1,QB,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,Zach Edey,79,226,138,22,1.0,C,5756880.0,NBA,0.366667,0.131788,5,C,3
2024,Zach LaVine,85,196,91,29,11.0,SG,43031940.0,NBA,0.566667,1.123926,2,G,1
2025,Zeke Nnaji,72,206,109,23,5.0,C,8888889.0,NBA,0.133333,-1.025706,5,C,3
2026,Ziaire Williams,76,206,84,22,4.0,SF,6133005.0,NBA,0.266667,-0.364281,3,F,2


In [245]:
nba_year = df_pro_years[ df_pro_years["sport"] == "NBA"] 
nfl_year = df_pro_years[ df_pro_years["sport"] == "NFL"]
nba = df_no_years [ df_no_years ["sport"] == "NBA"] 
nfl =df_no_years[ df_no_years["sport"] == "NFL"]
fifa = df_no_years [ df_no_years ["sport"] == "FIFA"] 

In [291]:
dataset = [
    (nba_year, "nba_year"),
    (nfl_year, "nfl_year"),
    (nba, "nba"),
    (nfl, "nfl"),
    #(fifa, "fifa")
]

## Building model

### Linear regression without position(hypothesis)

In [292]:
def APY_with_overall_linear_regression_model(dataset, overall_type="overall", years=None):
    
    dataset_name = dataset.__name__ if hasattr(dataset, "__name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[[overall_type, "height_cm", "weight_kg", "Age", "years_pro"]] 
    else:
        x = dataset[[overall_type, "height_cm", "weight_kg", "Age"]] 
        
    y = dataset["APY"]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    lin_model = LinearRegression()
    lin_model.fit(x_train, y_train)
    
    y_pred = lin_model.predict(x_test) 
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results(with overall):")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

In [293]:
def APY_without_overall_linear_regression_model(dataset, years=None):
    
    dataset_name = dataset.__name__ if hasattr(dataset, "__name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[[ "height_cm", "weight_kg", "Age", "years_pro"]] 
    else:
        x = dataset[[ "height_cm", "weight_kg", "Age"]] 
        
    y = dataset["APY"]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    lin_model = LinearRegression()
    lin_model.fit(x_train, y_train)
    
    y_pred = lin_model.predict(x_test) 
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results(without overall):")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

In [248]:
model_try = [APY_with_overall_linear_regression_model, APY_without_overall_linear_regression_model]

### baseline models(without positions)

#### Linear regression

##### basic

In [294]:
def overall_basic_linear_regression_model(dataset, overall_type="overall", years=None):
    dataset_name = dataset.__name__ if hasattr(dataset, "__name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "Age"]] 
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    lin_model = LinearRegression()
    lin_model.fit(x_train, y_train)
    
    y_pred = lin_model.predict(x_test) 
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

##### ridge

In [295]:
def overall_basic_ridge_model(dataset, overall_type="overall", years=None):
    dataset_name = dataset.__name__ if hasattr(dataset, "____name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "Age"]] 
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    }
    ridge = Ridge(random_state=2025)
    
    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
 
    y_pred = best_model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results (with tuned hyperparameters):")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

##### lasso

In [296]:
def overall_basic_lasso_model(dataset, overall_type="overall", years=None):
    dataset_name = dataset.__name__ if hasattr(dataset, "____name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "Age"]] 
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
        'selection': ['cyclic', 'random']
    }
    lasso = Lasso(random_state=2025, max_iter=10000)
    
    grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
 
    y_pred = best_model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results (with tuned hyperparameters):")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

#### RandomForestRegressor

In [290]:
def overall_basic_randomforest_regression_model(dataset, overall_type="overall", years=None):
    dataset_name = dataset.__name__ if hasattr(dataset, "__name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "Age"]] 
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    rf = RandomForestRegressor(random_state=2025)
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='r2',n_jobs=-1)
    grid_search.fit(x_train, y_train)
    best_rf = grid_search.best_estimator_
    y_pred = best_rf.predict(x_test) 
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results (with tuned hyperparameters):")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

### nn

In [297]:
baseline_model = [overall_basic_linear_regression_model,overall_basic_ridge_model,overall_basic_lasso_model]

### fancy cool stuff (with original position)

#### Linear regression

##### basic

In [312]:
def fullpos_overall_linear_regression_model(dataset, overall_type="overall", years=None):
    
    dataset_name = dataset.__name__ if hasattr(dataset, "__name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age","position_label", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "position_label","Age"]] 
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    lin_model = LinearRegression()
    lin_model.fit(x_train, y_train)
    
    y_pred = lin_model.predict(x_test) 
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

##### ridge

In [313]:
def fullpos_overall_basic_ridge_model(dataset, overall_type="overall", years=None):
    dataset_name = dataset.__name__ if hasattr(dataset, "____name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age","position_label", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "position_label","Age"]] 
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    }
    ridge = Ridge(random_state=2025)
    
    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
 
    y_pred = best_model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results (with tuned hyperparameters):")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

##### lasso

In [314]:
def fullpos_overall_basic_lasso_model(dataset, overall_type="overall", years=None):
    dataset_name = dataset.__name__ if hasattr(dataset, "____name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age","position_label", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "position_label","Age"]] 
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
        'selection': ['cyclic', 'random']
    }
    lasso = Lasso(random_state=2025, max_iter=10000)
    
    grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
 
    y_pred = best_model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results (with tuned hyperparameters):")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

#### RandomForestRegressor

In [315]:
def fullpos_overall_basic_randomforest_regression_model(dataset, overall_type="overall", years=None):
    dataset_name = dataset.__name__ if hasattr(dataset, "__name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age","position_label", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "position_label","Age"]]  
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    rf = RandomForestRegressor(random_state=2025)
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='r2',n_jobs=-1)
    grid_search.fit(x_train, y_train)
    best_rf = grid_search.best_estimator_
    y_pred = best_rf.predict(x_test) 
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results (with tuned hyperparameters):")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

In [316]:
fullpos_model=[fullpos_overall_linear_regression_model,fullpos_overall_basic_ridge_model,fullpos_overall_basic_lasso_model,fullpos_overall_basic_randomforest_regression_model]

### super cool(with simplified position)

#### Linear regression

##### basic

In [277]:
def simplifiedpos_overall_linear_regression_model(dataset, overall_type="overall", years=None):
    
    dataset_name = dataset.__name__ if hasattr(dataset, "__name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age","position_encode_simplified", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "position_encode_simplified","Age"]] 
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    lin_model = LinearRegression()
    lin_model.fit(x_train, y_train)
    
    y_pred = lin_model.predict(x_test) 
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

##### ridge

In [317]:
def simplifiedpos_overall_basic_ridge_model(dataset, overall_type="overall", years=None):
    dataset_name = dataset.__name__ if hasattr(dataset, "____name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age","position_encode_simplified", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "position_encode_simplified","Age"]]
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    }
    ridge = Ridge(random_state=2025)
    
    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
 
    y_pred = best_model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results (with tuned hyperparameters):")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

##### lasso

In [318]:
def simplifiedpos_overall_basic_lasso_model(dataset, overall_type="overall", years=None):
    dataset_name = dataset.__name__ if hasattr(dataset, "____name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age","position_encode_simplified", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "position_encode_simplified","Age"]]
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
        'selection': ['cyclic', 'random']
    }
    lasso = Lasso(random_state=2025, max_iter=10000)
    
    grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
 
    y_pred = best_model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results (with tuned hyperparameters):")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

#### RandomForestRegressor

In [319]:
def simplifiedpos_overall_basic_randomforest_regression_model(dataset, overall_type="overall", years=None):
    dataset_name = dataset.__name__ if hasattr(dataset, "__name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[["height_cm", "weight_kg", "Age","position_encode_simplified", "years_pro"]] 
    else:
        x = dataset[["height_cm", "weight_kg", "position_encode_simplified","Age"]]
        
    y = dataset[overall_type]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    rf = RandomForestRegressor(random_state=2025)
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='r2',n_jobs=-1)
    grid_search.fit(x_train, y_train)
    best_rf = grid_search.best_estimator_
    y_pred = best_rf.predict(x_test) 
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results (with tuned hyperparameters):")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

In [320]:
simple_pos_model=[simplifiedpos_overall_linear_regression_model,simplifiedpos_overall_basic_ridge_model,simplifiedpos_overall_basic_lasso_model,simplifiedpos_overall_basic_randomforest_regression_model]

In [275]:
df.head()

,name,overall,height_cm,weight_kg,Age,years_pro,position,APY,sport,overall_minmax,overall_standard,position_label,position_simplified,position_encode_simplified
0,Dak Prescott,87,188,108,30,7.0,QB,60000000.0,NFL,0.835616,1.582078,1,QB,1
1,Trevor Lawrence,82,198,100,23,2.0,QB,55000000.0,NFL,0.767123,1.082987,1,QB,1
2,Jordan Love,70,193,99,24,3.0,QB,55000000.0,NFL,0.602740,-0.114831,1,QB,1
3,Joe Burrow,95,193,98,26,3.0,QB,55000000.0,NFL,0.945205,2.380623,1,QB,1
4,Tua Tagovailoa,83,185,98,25,3.0,QB,53100000.0,NFL,0.780822,1.182805,1,QB,1


## Runing model

In [252]:
def run_model(datasets_with_names, models):
    results = []

    for dataset, dataset_name in datasets_with_names:

        for model_func in models:
            print(f"Running {model_func.__name__} on {dataset_name}")
            try:
                model_result = model_func(dataset)
                results.append({
                    "dataset": dataset_name,
                    "model": model_func.__name__,
                    "results": model_result
                })
                print("-" * 50)
            except Exception as e:
                print(f"Error running {model_func.__name__} on {dataset_name}: {e}")

### checking hypothesis

In [249]:
run_model(dataset, model_try)

Running APY_with_overall_linear_regression_model on nba_year
Model Results(with overall):
MSE: 48698969825719.6562
RMSE: 6978464.7184
R²: 0.5981
--------------------------------------------------
Running APY_without_overall_linear_regression_model on nba_year
Model Results(without overall):
MSE: 101907847455120.7656
RMSE: 10094941.6767
R²: 0.1590
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nfl_year
Model Results(with overall):
MSE: 18883384903311.2344
RMSE: 4345501.6860
R²: 0.3735
--------------------------------------------------
Running APY_without_overall_linear_regression_model on nfl_year
Model Results(without overall):
MSE: 24962081256429.3242
RMSE: 4996206.6867
R²: 0.1719
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nba
Model Results(with overall):
MSE: 48241489167503.4609
RMSE: 6945609.3446
R²: 0.6019
--------------------------------------------------
Running APY

In [213]:
def run_model_check(datasets_with_names, model_func, **model_kwargs):
    results = []
    for dataset, dataset_name in datasets_with_names:
        print(f"Running {model_func.__name__} on {dataset_name}")

        try:
            model_result = model_func(dataset, **model_kwargs)
            results.append({
                'dataset': dataset_name,
                'model': model_func.__name__,
                'results': model_result
            })
            
            print("-" * 50)
        except Exception as e:
            print(f"Error running {model_func.__name__} on {dataset_name}: {e}")
    

In [214]:
run_model_check(dataset, APY_with_overall_linear_regression_model, overall_type="overall_standard")

Running APY_with_overall_linear_regression_model on nba_year
Model Results(with overall):
MSE: 48698969825719.6875
RMSE: 6978464.7184
R²: 0.5981
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nfl_year
Model Results(with overall):
MSE: 18883384903311.2383
RMSE: 4345501.6860
R²: 0.3735
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nba
Model Results(with overall):
MSE: 48241489167503.4844
RMSE: 6945609.3446
R²: 0.6019
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nfl
Model Results(with overall):
MSE: 19441712628964.9375
RMSE: 4409275.7488
R²: 0.3550
--------------------------------------------------
Running APY_with_overall_linear_regression_model on fifa
Model Results(with overall):
MSE: 694354464833.3361
RMSE: 833279.3438
R²: 0.3889
--------------------------------------------------


In [215]:
run_model_check(dataset, APY_with_overall_linear_regression_model, overall_type="overall_minmax")

Running APY_with_overall_linear_regression_model on nba_year
Model Results(with overall):
MSE: 48698969825719.6719
RMSE: 6978464.7184
R²: 0.5981
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nfl_year
Model Results(with overall):
MSE: 18883384903311.2461
RMSE: 4345501.6860
R²: 0.3735
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nba
Model Results(with overall):
MSE: 48241489167503.4531
RMSE: 6945609.3446
R²: 0.6019
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nfl
Model Results(with overall):
MSE: 19441712628964.9492
RMSE: 4409275.7488
R²: 0.3550
--------------------------------------------------
Running APY_with_overall_linear_regression_model on fifa
Model Results(with overall):
MSE: 694354464833.3359
RMSE: 833279.3438
R²: 0.3889
--------------------------------------------------


### performance

In [300]:
run_model(dataset,baseline_model)

Running overall_basic_linear_regression_model on nba_year
Model Results:
MSE: 28.7802
RMSE: 5.3647
R²: 0.1023
--------------------------------------------------
Running overall_basic_ridge_model on nba_year
Model Results (with tuned hyperparameters):
Best parameters: {'alpha': 10.0, 'solver': 'saga'}
MSE: 28.7996
RMSE: 5.3665
R²: 0.1017
--------------------------------------------------
Running overall_basic_lasso_model on nba_year
Model Results (with tuned hyperparameters):
Best parameters: {'alpha': 0.1, 'selection': 'random'}
MSE: 28.7813
RMSE: 5.3648
R²: 0.1023
--------------------------------------------------
Running overall_basic_linear_regression_model on nfl_year
Model Results:
MSE: 71.6104
RMSE: 8.4623
R²: 0.1924
--------------------------------------------------
Running overall_basic_ridge_model on nfl_year
Model Results (with tuned hyperparameters):
Best parameters: {'alpha': 10.0, 'solver': 'saga'}
MSE: 71.3438
RMSE: 8.4465
R²: 0.1954
--------------------------------------

In [308]:
run_model(dataset,fullpos_model)

Running fullpos_overall_linear_regression_model on nba_year
Model Results:
MSE: 28.4307
RMSE: 5.3320
R²: 0.1132
--------------------------------------------------
Running fullpos_overall_basic_ridge_model on nba_year
Model Results (with tuned hyperparameters):
Best parameters: {'alpha': 10.0, 'solver': 'saga'}
MSE: 28.4398
RMSE: 5.3329
R²: 0.1129
--------------------------------------------------
Running fullpos_overall_basic_lasso_model on nba_year
Model Results (with tuned hyperparameters):
Best parameters: {'alpha': 0.1, 'selection': 'cyclic'}
MSE: 28.5002
RMSE: 5.3386
R²: 0.1110
--------------------------------------------------
Running fullpos_overall_basic_randomforest_regression_model on nba_year
Model Results (with tuned hyperparameters):
Best parameters: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
MSE: 30.2888
RMSE: 5.5035
R²: 0.0552
--------------------------------------------------
Running fullpos_overall_linear_regression_model on n

In [321]:
run_model(dataset,simple_pos_model)

Running simplifiedpos_overall_linear_regression_model on nba_year
Model Results:
MSE: 28.2260
RMSE: 5.3128
R²: 0.1196
--------------------------------------------------
Running simplifiedpos_overall_basic_ridge_model on nba_year
Model Results (with tuned hyperparameters):
Best parameters: {'alpha': 10.0, 'solver': 'sag'}
MSE: 28.2838
RMSE: 5.3182
R²: 0.1178
--------------------------------------------------
Running simplifiedpos_overall_basic_lasso_model on nba_year
Model Results (with tuned hyperparameters):
Best parameters: {'alpha': 0.1, 'selection': 'random'}
MSE: 28.4201
RMSE: 5.3310
R²: 0.1135
--------------------------------------------------
Running simplifiedpos_overall_basic_randomforest_regression_model on nba_year
Model Results (with tuned hyperparameters):
Best parameters: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
MSE: 30.7731
RMSE: 5.5473
R²: 0.0401
--------------------------------------------------
Running simplifiedpos_overall

In [323]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def nn_regression_model(dataset, overall_type="overall", years=None):
    has_years_pro = "years_pro" in dataset.columns
    
    # 选择特征
    if has_years_pro:
        features = ["height_cm", "weight_kg", "Age", "position_encode_simplified", "years_pro"]
    else:
        features = ["height_cm", "weight_kg", "Age", "position_encode_simplified"]
    
    X = dataset[features]
    y = dataset[overall_type]
    
    # 数据分割
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2025, test_size=0.2)
    
    # 标准化特征
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 构建模型
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    
    # 编译模型
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    
    # 早停策略
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    
    # 训练模型
    model.fit(
        X_train_scaled, y_train,
        epochs=200,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # 预测
    y_pred = model.predict(X_test_scaled).flatten()
    
    # 计算指标
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Neural Network Results:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    
    return {
        'model': model,
        'scaler': scaler,
        'metrics': {
            'mse': mse,
            'rmse': rmse,
            'r2': r2
        }
    }

In [324]:
x=[nn_regression_model]

In [325]:
run_model(dataset,x)

Running nn_regression_model on nba_year


2025-04-03 20:00:25.151686: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:/opt/slurm/lib64:
2025-04-03 20:00:25.151712: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-04-03 20:00:25.151728: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0702a-s2.ufhpc): /proc/driver/nvidia/version does not exist
2025-04-03 20:00:25.152627: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Neural Network Results:
MSE: 30.4775
RMSE: 5.5206
R²: 0.0494
--------------------------------------------------
Running nn_regression_model on nfl_year
Neural Network Results:
MSE: 63.9077
RMSE: 7.9942
R²: 0.2792
--------------------------------------------------
Running nn_regression_model on nba
Neural Network Results:
MSE: 31.2741
RMSE: 5.5923
R²: 0.0245
--------------------------------------------------
Running nn_regression_model on nfl
Neural Network Results:
MSE: 67.2960
RMSE: 8.2034
R²: 0.2410
--------------------------------------------------


In [329]:
df[df['sport']=='nba']

,name,overall,height_cm,weight_kg,Age,years_pro,position,APY,sport,overall_minmax,overall_standard,position_label,position_simplified,position_encode_simplified
0,Dak Prescott,87,188,108,30,7.0,QB,60000000.0,nba,0.835616,1.582078,1,QB,1
1,Trevor Lawrence,82,198,100,23,2.0,QB,55000000.0,nba,0.767123,1.082987,1,QB,1
2,Jordan Love,70,193,99,24,3.0,QB,55000000.0,nba,0.602740,-0.114831,1,QB,1
3,Joe Burrow,95,193,98,26,3.0,QB,55000000.0,nba,0.945205,2.380623,1,QB,1
4,Tua Tagovailoa,83,185,98,25,3.0,QB,53100000.0,nba,0.780822,1.182805,1,QB,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,Umut Yaşar Keçeci,58,182,73,19,NaN,ST,104000.0,nba,0.250000,-1.150147,0,0,0
15556,Rakan Al Kaabi,54,174,71,20,NaN,"CDM, CM",104000.0,nba,0.159091,-1.702407,0,0,0
15557,Rodrigo Frutos,58,188,87,20,NaN,GK,26000.0,nba,0.250000,-1.150147,0,0,0
15558,Christian Bos,55,180,70,18,NaN,RB,26000.0,nba,0.181818,-1.564342,0,0,0
